# Regression Transformer Model
Predicts the number of comments an article will get based on the article's topic, tone, length, hour of the day and day of the week

In [57]:
import tensorflow as tf
from keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense, Flatten
from keras.datasets import imdb
from keras.models import Sequential, Model
import pandas as pd
import numpy as np
import warnings

In [58]:
def transformer_block(embed_dim, num_heads, ff_dim, dropout_rate):
    att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    ffn = Sequential(
        [Dense(ff_dim, activation="relu"),
         Dense(embed_dim), ]
    )
    layernorm1 = LayerNormalization(epsilon=1e-6)
    layernorm2 = LayerNormalization(epsilon=1e-6)
    dropout1 = Dropout(dropout_rate)
    dropout2 = Dropout(dropout_rate)
    
    def call(inputs, training):
        attn_output = att(inputs, inputs)
        attn_output = dropout1(attn_output, training=training)
        out1 = layernorm1(inputs + attn_output)
        ffn_output = ffn(out1)
        ffn_output = dropout2(ffn_output, training=training)
        return layernorm2(out1 + ffn_output)
    
    return call

In [59]:
#Load training data
train_df = pd.read_csv('../data/regression_train_data.csv')

# Extract the columns you want to use as input features
columns = ['BERT_sentiment_score', 'normalised_word_count', 'pub_day', 'pub_hour', 'topic']
# Extract input features from the dataframe
x_train = np.array(train_df[columns])

y_train = np.array(train_df['n_comments'])

In [60]:
# Training and validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [61]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_mae', 
                               # Number of epochs to wait for improvement
                               patience=10,  
                               verbose=1, 
                               # Restore the weights of the best epoch
                               restore_best_weights=True)

In [62]:
# Hypreparameters
max_seq_length = x_train.shape[1]
vocab_size = int(np.max(x_train) + 1)
embedding_dim = 32
num_heads = 2
ff_dim = 32
dropout_rate = 0.1

In [63]:
# Define model architecture

# Input
inputs = Input(shape=(max_seq_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length)(inputs)

# Hidden layers
transformer_block_fn = transformer_block(embed_dim=embedding_dim, 
                                         num_heads=num_heads, ff_dim=ff_dim, 
                                         dropout_rate=dropout_rate)
transformer_block = transformer_block_fn(embedding_layer, training=True)
pooling_layer = GlobalAveragePooling1D()(transformer_block)
dropout_layer = Dropout(rate=0.1)(pooling_layer)

# Output
outputs = Dense(units=1, activation='linear')(dropout_layer)  # Change activation to 'linear' for regression

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 5, 32)        2688        ['input_3[0][0]']                
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 5, 32)       8416        ['embedding_2[0][0]',            
 eadAttention)                                                    'embedding_2[0][0]']            
                                                                                                  
 dropout_6 (Dropout)            (None, 5, 32)        0           ['multi_head_attention_2[0]

In [65]:
# Compile the model
model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])  # Change loss function to 'mse' for regression

# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=100, validation_data=(x_val, y_val), 
          callbacks=[early_stopping])

Epoch 1/100
125/125 [==============================] - 2s 9ms/step - loss: 373938.5938 - mae: 302.9965 - val_loss: 321037.4375 - val_mae: 274.5068
Epoch 2/100
125/125 [==============================] - 1s 7ms/step - loss: 368464.0000 - mae: 297.4241 - val_loss: 315376.8125 - val_mae: 269.0971
Epoch 3/100
125/125 [==============================] - 1s 6ms/step - loss: 361220.4688 - mae: 292.0836 - val_loss: 307846.7812 - val_mae: 264.0556
Epoch 4/100
125/125 [==============================] - 1s 6ms/step - loss: 352247.7500 - mae: 287.1567 - val_loss: 299199.7188 - val_mae: 260.3413
Epoch 5/100
125/125 [==============================] - 1s 6ms/step - loss: 342388.6562 - mae: 283.8164 - val_loss: 290011.8125 - val_mae: 258.2507
Epoch 6/100
125/125 [==============================] - 1s 6ms/step - loss: 332039.6562 - mae: 281.6322 - val_loss: 280753.1562 - val_mae: 257.5047
Epoch 7/100
125/125 [==============================] - 1s 6ms/step - loss: 321522.1250 - mae: 277.8135 - val_loss: 271

In [66]:
# save the trained model
model.save('../models/transformer_regression_model.h5')

In [68]:
# Evaluate the model on validation data
mse, mae = model.evaluate(x_val, y_val)
print('Validation MSE:', mse)
print('Validation MAE:', mae)

63/63 [==============================] - 0s 2ms/step - loss: 173913.9219 - mae: 223.2462
Validation MSE: 173913.921875
Validation MAE: 223.24618530273438
